In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# import cudf as pd
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
# nltk.download("wordnet")
# nltk.download('punkt')
# nltk.download("omw-1.4")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v2.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v1.csv
/kaggle/input/argugpt/argugpt.csv
/kaggle/input/argugpt/machine-dev.csv
/kaggle/input/argugpt/machine-test.csv
/kaggle/input/argugpt/machine-train.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv
/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Check if a GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Check TensorFlow version and GPU availability
print("TensorFlow Version: ", tf.__version__)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
TensorFlow Version:  2.13.0
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [5]:
# Dataset from the competition
train_essays_base = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
test_essays_base = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
train_prompts_base = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')

# Dataset from argugpt
argu_base = pd.read_csv('/kaggle/input/argugpt/argugpt.csv')

# # Dataset from daigt
# daigt_1_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
# daigt_2_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv')
# daigt_3_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv')
# daigt_4_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')

# Dataset from daigt v2
daigtv2_base = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')

# Dataset from Rdizzl3
llm_base = pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v2.csv')

In [6]:
train_esssays = train_essays_base[['text', 'generated']]
# train_esssays['generated'] = 0

In [7]:
argu = argu_base[['text']]
argu['generated'] = 1

/tmp/ipykernel_26/2722883321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  argu['generated'] = 1


In [8]:
# daigt_1 = daigt_1_base[['text', 'label']]
# daigt_1.rename(columns={'label':'generated'}, inplace=True)
# daigt_2 = daigt_2_base[['text', 'label']]
# daigt_2.rename(columns={'label':'generated'}, inplace=True)
# daigt_3 = daigt_3_base[['text', 'label']]
# daigt_3.rename(columns={'label':'generated'}, inplace=True)
# daigt_4 = daigt_4_base[['text', 'label']]
# daigt_4.rename(columns={'label':'generated'}, inplace=True)

In [9]:
train_ = daigtv2_base[daigtv2_base.RDizzl3_seven == False].reset_index(drop=True)
train_ = daigtv2_base[daigtv2_base["label"]==1].sample(8000)
daigtv2_base = daigtv2_base[daigtv2_base.RDizzl3_seven == True].reset_index(drop=True)
daigtv2_base = pd.concat([daigtv2_base, train_])

daigtv2 = daigtv2_base[['text', 'label']]
daigtv2.rename(columns={'label':'generated'}, inplace=True)

/tmp/ipykernel_26/3126248357.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daigtv2.rename(columns={'label':'generated'}, inplace=True)


In [10]:
llm = llm_base[['text', 'label']]
llm.rename(columns={'label':'generated'}, inplace=True)

In [11]:
# argu
# train_essays
# daigt_1
df = pd.concat([
    train_esssays,
    argu,
    llm,
    daigtv2
])

df['text'] = df['text'].str.replace('\n', ' ')
df['text'] = df['text'].str.replace('\"', '')
df['text'] = df['text'].str.replace('\'', '')
df['text'] = df['text'].str.replace(',', ' ')
df['text'] = df['text'].str.replace('?', ' ')
df['text'] = df['text'].str.replace('!', ' ')
df['text'] = df['text'].str.replace('\\', '')
df['text'] = df['text'].str.replace('\"', '')
df['text'] = df['text'].str.replace(r'<[^>]+>', '', regex=True)
df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True)

In [12]:
df = df.dropna()
df['generated'].value_counts()

generated
0    29872
1    21245
Name: count, dtype: int64

In [13]:
NUMBER_OF_SAMPLES = 21000
NUMBER_OF_MODELS = 4

In [14]:

negative = df[df['generated'] == 0]
positive = df[df['generated'] == 1]
df = pd.concat([negative.sample(NUMBER_OF_SAMPLES), positive.sample(NUMBER_OF_SAMPLES)])
df = df.sample(NUMBER_OF_SAMPLES)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21000 entries, 5632 to 9647
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       21000 non-null  object
 1   generated  21000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 492.2+ KB


In [15]:
# import gensim
# from gensim.models import Word2Vec

In [16]:
class EnsembleNN():

    def __init__(self, number_of_samples, number_of_models):
        self.nos = number_of_samples
        self.nom = number_of_models
        self.df = None
        self.models = []
        self.get_optimizer()

    def insert_df(self, df, x_column, y_column, test):
        print('Inserting df')
        self.df = df
        self.x = x_column
        self.y = y_column
        if test is not None:
            self.generate_vectorizer(test)
        else:
            self.generate_vectorizer(df)
#         self.generate_word2vec()
        
    def generate_word2vec(self, vector_size=100, window=5, min_count=1, workers=4):
        # Assuming your data is already tokenized, if not, you may need to tokenize it first.
        sentences = [text.split() for text in self.df[self.x]]
        self.word2vec_model = Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

    def generate_vectorizer(self,df, max_feature=None, n_grams=5):
        print('TFIDF vectorizer')
        tfidf_vectorizer = TfidfVectorizer(max_features=max_feature,
                                           lowercase=True,
                                           analyzer='word',
                                           stop_words='english',
                                           ngram_range=(1, n_grams),
                                           sublinear_tf=True)
        self.tfidf = tfidf_vectorizer.fit(df[self.x])
        print('TFIDF finished!')
        
    def word2vec_transform(self, input):
        # Transform a sentence into the averaged Word2Vec representation
        vectors = [np.mean([self.word2vec_model.wv[word] for word in sentence.split() if word in self.word2vec_model.wv]
                           or [np.zeros(self.word2vec_model.vector_size)], axis=0)
                   for sentence in input]
        return np.vstack(vectors)
        
    def generate_models(self):
        if gpus:
#             try:
#                 # Allow GPU memory growth
#                 for gpu in gpus:
#                     tf.config.experimental.set_memory_growth(gpu, True)
#             except RuntimeError as e:
#                 print(e)
            # Explicitly specify the GPU device
            with tf.device('/GPU:0'):
                for i in range(self.nom):
                    self.create_model()
                print("GPU is available and used.")
        else:
            print("No GPU available.")
        
    def tfidf_transform(self, input):
        return self.tfidf.transform(input)
    
    def get_optimizer(self):
        # Define the learning rate schedule
        initial_learning_rate = 1e-2
        decay_steps = 10000
        end_learning_rate = 1e-5
        power = 1.0

        lr_schedule = PolynomialDecay(
            initial_learning_rate,
            decay_steps,
            end_learning_rate,
            power
        )

        # Use the AdamW optimizer with weight decay
        self.optimizer = Adam(learning_rate=lr_schedule, epsilon=1e-08, clipvalue=0.5, amsgrad=False, name='AdamW')
    
    def create_model(self):
        if self.nos is None:
            self.nos = len(self.df)
        samples = self.df.sample(self.nos)
        train_x, test_x, train_y, test_y = train_test_split(samples[self.x], samples[self.y], test_size=0.3)
        train_x = self.tfidf.transform(train_x).toarray()
#         train_x = self.word2vec_transform(train_x)
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(2048, activation='relu'),
            tf.keras.layers.Dense(1024, activation='relu'),
            tf.keras.layers.Dropout(0.95),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(train_x, train_y, epochs=16, batch_size=64, validation_split=0.1)

        test_x = self.tfidf.transform(test_x).toarray()
#         test_x = self.word2vec_transform(test_x)
        loss, accuracy = model.evaluate(test_x, test_y)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
        self.models.append(model)

    def predict(self, test):
        predictions = None
        for m in self.models:
            p = m.predict(test)
            if predictions is not None:
                predictions = np.add(p, predictions)
            else:
                predictions = p

        return predictions/self.nom

In [17]:
enn = EnsembleNN(NUMBER_OF_SAMPLES, NUMBER_OF_MODELS)
enn.insert_df(df, 'text', 'generated', test_essays_base)

Inserting df
TFIDF vectorizer
TFIDF finished!


In [18]:
enn.generate_models()

Epoch 1/16
207/207 [==============================] - 6s 4ms/step - loss: 0.6931 - accuracy: 0.5090 - val_loss: 0.6936 - val_accuracy: 0.4789
Epoch 2/16
207/207 [==============================] - 1s 3ms/step - loss: 0.6930 - accuracy: 0.5090 - val_loss: 0.6939 - val_accuracy: 0.4789
Epoch 3/16
207/207 [==============================] - 1s 4ms/step - loss: 0.6930 - accuracy: 0.5090 - val_loss: 0.6939 - val_accuracy: 0.4789
Epoch 4/16
207/207 [==============================] - 1s 4ms/step - loss: 0.6930 - accuracy: 0.5090 - val_loss: 0.6940 - val_accuracy: 0.4789
Epoch 5/16
207/207 [==============================] - 1s 4ms/step - loss: 0.6930 - accuracy: 0.5090 - val_loss: 0.6941 - val_accuracy: 0.4789
Epoch 6/16
207/207 [==============================] - 1s 4ms/step - loss: 0.6930 - accuracy: 0.5090 - val_loss: 0.6941 - val_accuracy: 0.4789
Epoch 7/16
207/207 [==============================] - 1s 4ms/step - loss: 0.6930 - accuracy: 0.5090 - val_loss: 0.6940 - val_accuracy: 0.4789
Epoch 

In [19]:
# if gpus:
#     try:
#         # Allow GPU memory growth
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
# #             break

#         # Explicitly specify the GPU device
#         with tf.device('/GPU:0'):
#             # Your TensorFlow code here
#             # This block will run on the GPU
#             model.fit(train_x, train_y, epochs=8, batch_size=64, validation_split=0.1)
#             print("GPU is available and used.")
            
#     except RuntimeError as e:
#         print(e)
# else:
#     print("No GPU available.")


In [20]:
# essay_test = enn.tfidf_transform(test_x).toarray()
# loss, accuracy = model.evaluate(essay_test, test_y)
# print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

In [21]:
# # Make predictions
# predictions = model.predict(essay_test)

# # Evaluate accuracy
# accuracy = accuracy_score(test_y, predictions)
# print(f'Accuracy: {accuracy:.2f}')

# # Display classification report
# print(classification_report(test_y, predictions))

In [22]:
test_essays = enn.tfidf_transform(test_essays_base['text']).toarray()

In [23]:
res = enn.predict(test_essays)

1/1 [==============================] - 0s 62ms/step


In [24]:
submission = test_essays_base[['id']]
submission['generated'] = res

/tmp/ipykernel_26/4081722939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['generated'] = res


In [25]:
submission['generated'] = submission['generated'].apply(lambda a: (int(a)*10 + int(((a%1)*10)))/10)

/tmp/ipykernel_26/3021702422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['generated'] = submission['generated'].apply(lambda a: (int(a)*10 + int(((a%1)*10)))/10)


In [26]:
submission.to_csv('submission.csv', index=False)